In [5]:
import os
import sys
import importlib

import pandas as pd
import numpy as np

from scipy.io import mmread

sys.path.append(os.path.abspath("./src"))

import utility as ut
import hessenberg as hg
from variables import *
from qr import QR


pd.set_option("display.max_columns", 10)
pd.set_option("display.width", 1000)
pd.set_option("display.precision", 12)
print("Setup compelete.")


Setup compelete.


# Hessenberg Transform

## Complex Matrices

In [6]:
a = -20
b = 50
n = 5
m = ut.complex_matrix(n, a, b)
print(f"Original matrix:\n {pd.DataFrame(m)}")
h, u = hg.hessenberg_transform(m) 
print(f"Hessenberg transformed:\n {pd.DataFrame(h)}")
print(f"Transformation matrix:\n {pd.DataFrame(u)}")
print(f"Is the transformation similar: {np.allclose(u @ h @ u.conj().T - m, np.zeros((n, n)))}")

Original matrix:
                                   0                                 1                                 2                                 3                                 4
0  35.081102952181-14.503119257712j   7.492552258600+42.230223168994j  3.2552069067760-8.2920810574570j  48.062449551394+37.107064156168j -17.699048962012+15.110463720002j
1  42.236003469685-18.918366260403j  23.292840548227+31.699577771487j  -7.174232966044-12.549994806711j  47.060925873227+39.412051378583j   8.391077597892+19.625075057113j
2  36.938364079874+17.891185210745j  45.873932812867+32.426063634327j  8.7351289887530+4.0309748596360j   26.959678797315-3.632529866017j -18.218495628804+31.310671932309j
3  45.908791284326+23.571433086131j -4.5621928564720+2.1940346896320j  13.282849676561-14.155139512640j   0.861014025491+31.616677708143j   7.429400586139+46.969967010896j
4  3.8529030244730+1.8287843157240j  35.165184565453+47.600896229660j   37.123395691276-5.557139489932j  41.203910426911-1

## Real Matrices

### Random Matrices

In [7]:
a = -20
b = 50
n = 5
m = (b - a) * np.random.default_rng().random((n, n)) + a
print(f"Original matrix:\n {pd.DataFrame(m)}")
h, u = hg.hessenberg_transform(m) 
print(f"Hessenberg transformed:\n {pd.DataFrame(h)}")
print(f"Transformation matrix:\n {pd.DataFrame(u)}")
print(f"Is the transformation similar: {np.allclose(u @ h @ u.T - m, np.zeros((n, n)))}")

Original matrix:
                  0                1                2                3                4
0 -15.082149966568 -18.309761712307  -0.993315390544 -11.272623823717  44.228565760211
1 -14.746541589804  33.674336676788   7.053615075260  26.505213223707  24.454029621142
2  23.642520516414   2.660071027561  11.410001620909  -4.361025290479  20.277355838021
3  33.184909568800  -0.968275909089   4.109998484614  -7.962732143947   0.604157091100
4  21.974784692510  11.436502973577  29.010232280062  41.040051667315  48.421842983972
Hessenberg transformed:
                  0                1                2                3                4
0 -15.082149966568  17.378682704068 -30.718935660567  31.529700257650 -13.404421192984
1  48.585580686048  23.968300085453 -16.872178156269   0.674812256323 -12.475208453712
2   0.000000000000 -61.022151903569  38.391086863229  22.454284937598 -24.747426275829
3   0.000000000000   0.000000000000  19.846174313935  12.336338480745  -7.656197379370


### Matrix Market

In [8]:
files = ["west0381", "blckhole"]
for file in files:
	mat = mmread(os.path.join("./test_matrices", ".".join([file, MATRIX_MARKET_FILE_EXT])))
	m = mat.toarray()
	h, u = hg.hessenberg_transform(m) 
	print(f"Is the transformation similar: {np.allclose(u @ h @ u.T - m, np.zeros((m.shape[0], m.shape[0])))}")

Is the transformation similar: True
Is the transformation similar: True


# QR

## Wilkinson Shift

### Complex Matrices

In [9]:
a = -20
b = 50
n = 5
tol = 1e-8
m = ut.complex_matrix(n, a, b)
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

                 Eigenvalues (Numpy)
0  93.838136591486+103.890445536860j
1   60.914275787707-16.778380678501j
2   5.8898025504440+7.0813736084130j
3  -27.573096162295+44.759021339438j
4  -51.414229798658+13.075931746314j
                Eigenvalues (Script)
0  93.838136591486+103.890445536860j
1   60.914275787707-16.778380678501j
2   5.8898025504440+7.0813736084130j
3  -27.573096162295+44.759021339438j
4  -51.414229798658+13.075931746314j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


### Real Matrices

#### Random Matrices

In [10]:
a = -20
b = 50
n = 5
tol = 1e-8
m = (b - a) * np.random.default_rng().random((n, n)) + a
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

                Eigenvalues (Numpy)
0  101.583953596424-0.000000000000j
1   48.322175210398+0.000000000000j
2   7.078982918766+19.720776445445j
3   7.078982918766-19.720776445445j
4  -51.050608396021-0.000000000000j
               Eigenvalues (Script)
0  101.583953596424+0.000000000000j
1   48.322175210398+0.000000000000j
2   7.078982918766+19.720776445445j
3   7.078982918766-19.720776445445j
4  -51.050608396021+0.000000000000j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


In [11]:
m = np.array([[7, 3, 4, -11, -9, -2],
     [-6, 4, -5, 7, 1, 12],
     [-1, -9, 2, 2, 9, 1],
     [-8, 0, -1, 5, 0, 8],
     [-4, 3, -5, 7, 2, 10],
     [6, 1, 4, -11, -7, -1]], dtype = np.float64)
tol = 1e-8
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

   Eigenvalues (Numpy)
0             5.0+6.0j
1             5.0-6.0j
2             4.0+0.0j
3             3.0+0.0j
4             1.0+2.0j
5             1.0-2.0j
   Eigenvalues (Script)
0              5.0+6.0j
1              5.0-6.0j
2              4.0+0.0j
3              3.0+0.0j
4              1.0+2.0j
5              1.0-2.0j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


#### Matrix Market

In [14]:
files = ["gre__115"]
for file in files:
	mat = mmread(os.path.join("./test_matrices", ".".join([file, MATRIX_MARKET_FILE_EXT])))
	m = mat.toarray()
	tol = 1e-8
	qr_alg = QR(m)
	u, r = qr_alg.qr_wilkinson_shift(1e-128, 200)
	eigs = np.sort(np.linalg.eig(qr_alg.H)[0])[::-1]
	eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
	b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
	print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
	if not b:
		with open("output_mm.txt", "w") as f:
			f.write(f"{mm.to_string()}")
	
	
	# with open("output_mm.txt", "w") as f:
	# 	f.write(f"{pd.DataFrame(np.vstack([eigs_real, eigs_extracted, eigs_real - eigs_extracted]).T, columns = ['Eigenvalues (Numpy)', 'Eigenvalues (Script)', 'Difference']).to_string()}")

Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: False


## Double Shift

In [49]:
m = np.array([[7, 3, 4, -11, -9, -2],
     [-6, 4, -5, 7, 1, 12],
     [-1, -9, 2, 2, 9, 1],
     [-8, 0, -1, 5, 0, 8],
     [-4, 3, -5, 7, 2, 10],
     [6, 1, 4, -11, -7, -1]], dtype = np.float64)

qr_alg = QR(m)
u, r = qr_alg.double_shift(1e-128, 500)
# print(f"{pd.DataFrame(r)}")
eigs = np.sort(np.diag(r))[::-1]
eigs_real = np.sort(np.linalg.eig(qr_alg.H)[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"eigs real:\n {pd.DataFrame(eigs_real)}")
print(f"eigs approximated:\n {pd.DataFrame(eigs_extracted)}")

ValueError: M must be square and of shape (2, 2) but given shape is (3, 3).

In [49]:
all([True, True])

True